In [1]:
import os
import random
import pandas as pd
import re
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image
import requests
from PIL import Image
import numpy as np
from io import BytesIO
import cv2
import matplotlib.pyplot as plt

In [2]:
entity_unit_map = {
    'width': ['centimetre', 'cm', 'foot', 'ft', 'inch', 'in', 'metre', 'm', 'millimetre', 'mm', 'yard', 'yd'],
    'depth': ['centimetre', 'cm', 'foot', 'ft', 'inch', 'in', 'metre', 'm', 'millimetre', 'mm', 'yard', 'yd'],
    'height': ['centimetre', 'cm', 'foot', 'ft', 'inch', 'in', 'metre', 'm', 'millimetre', 'mm', 'yard', 'yd'],
    'item_weight': ['gram', 'g', 'kilogram', 'kg', 'microgram', 'µg', 'milligram', 'mg', 'ounce', 'oz', 'pound', 'lb', 'ton'],
    'maximum_weight_recommendation': ['gram', 'g', 'kilogram', 'kg', 'microgram', 'µg', 'milligram', 'mg', 'ounce', 'oz', 'pound', 'lb', 'ton', 't'],
    'voltage': ['kilovolt', 'kV', 'millivolt', 'mV', 'volt', 'V'],
    'wattage': ['kilowatt', 'kW', 'watt', 'W'],
    'item_volume': ['centilitre', 'cL', 'cubic foot', 'ft³', 'cubic inch', 'in³', 'cup', 'cup', 'decilitre', 'dL', 'fluid ounce', 'fl oz', 'gallon', 'gal', 'imperial gallon', 'imp gal', 'litre', 'L', 'microlitre', 'µL', 'millilitre', 'mL', 'pint', 'pt', 'quart', 'qt']
}


In [3]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np

class ImageDataGenerator:
    def __init__(self, batch_size, image_links, image_shape=None):
        self.batch_size = batch_size
        self.image_links = image_links
        self.image_shape = image_shape

    def __len__(self):
        return len(self.image_links) // self.batch_size

    def __getitem__(self, idx):
        batch_links = self.image_links[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch = []
        for link in batch_links:
            try:
                response = requests.get(link)
                response.raise_for_status()  # Raise an exception for HTTP errors
                image = Image.open(BytesIO(response.content))
                grayscale_image = image.convert('L')
                image_array = np.array(grayscale_image)
                image_array = cv2.resize(image_array,self.image_shape)

                # Optionally, validate the shape of the image
                if self.image_shape is not None:
                    if image_array.shape[:2] != self.image_shape[:2]:
                        raise ValueError(f"Image shape {image_array.shape} does not match the expected shape {self.image_shape}")

                batch.append(image_array)
            except Exception as e:
                print(f"Error loading image from {link}: {e}")

        return np.array(batch)


In [4]:
# Load train and test data
train_df = pd.read_csv('./dataset/train.csv')
test_df = pd.read_csv('./dataset/test.csv')


train_image_links = train_df['image_link'].tolist()
test_image_links = test_df['image_link'].tolist()

In [5]:
image_shape = (1024, 1024)
batch_size = 16

train_generator = ImageDataGenerator(batch_size, train_image_links, image_shape)
test_generator = ImageDataGenerator(batch_size, test_image_links, image_shape)

In [6]:
test_df

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth
...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight


In [7]:
import cv2
import numpy as np
# import pytesseract
# from pytesseract import Output
import easyocr
# from spellchecker import SpellChecker

In [91]:
def preprocess_image(image):
    # Noise Reduction
    denoised = cv2.medianBlur(image, 3)

    # Apply binary thresholding
    _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Resize image for better OCR results
    resized = cv2.resize(binary, (0, 0), fx=2, fy=2) 
    
    return resized

def apply_ocr(image):
    # Tesseract OCR
    config = '--oem 1 --psm 6'
    data = pytesseract.image_to_string(image, config=config)
    return data

def easyocr_ocr(image):
    # EasyOCR accepts the image directly instead of a file path
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image, detail=0)
    return " ".join(result)

def extract_value_and_unit_for_entity(text, entity):
    if entity not in entity_unit_map:
        raise ValueError(f"Entity '{entity}' not found in the entity_unit_map")
    
    # Get the relevant units for the entity
    relevant_units = entity_unit_map[entity]

    # Regular expression to match numeric values and units
    pattern = r'(\d+(\.\d+)?)\s*(\w+)'
    matches = re.findall(pattern, text)
    
    # Filter matches based on relevant units
    results = [f"{match[0]} {match[2]}" for match in matches if match[2].lower() in relevant_units]
    
    return results

# Main Function to run the OCR Pipeline with the image directly
def ocr_pipeline(image,entity_name):
    preprocessed_image = preprocess_image(image)
    fallback_result = easyocr_ocr(preprocessed_image)
    result_oc = extract_value_and_unit_for_entity(fallback_result,entity_name)
    if len(result_oc)!=0:
           return result_oc[0]


In [8]:
import cv2
# import pytesseract
import easyocr
from concurrent.futures import ThreadPoolExecutor

reader = easyocr.Reader(['en'], gpu=True)  # Reuse the reader with GPU enabled

def preprocess_image(image):
    # Noise Reduction
    denoised = cv2.medianBlur(image, 3)
    # Apply binary thresholding
    _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Resize image for better OCR results (optional: reduce scaling factor if performance is slow)
    resized = cv2.resize(binary, (0, 0), fx=1.5, fy=1.5)
    return resized

def apply_ocr(image):
    config = '--oem 1 --psm 6'
    return pytesseract.image_to_string(image, config=config)

def easyocr_ocr(image):
    result = reader.readtext(image, detail=0)
    return " ".join(result)

def extract_value_and_unit_for_entity(text, entity):
    if entity not in entity_unit_map:
        raise ValueError(f"Entity '{entity}' not found in the entity_unit_map")
    
    relevant_units = entity_unit_map[entity]
    pattern = r'(\d+(\.\d+)?)\s*(\w+)'
    matches = re.findall(pattern, text)
    results = [f"{match[0]} {match[2]}" for match in matches if match[2].lower() in relevant_units]
    return results

def ocr_pipeline(image, entity_name):
    preprocessed_image = preprocess_image(image)
    easyocr_result = easyocr_ocr(preprocessed_image)
    extracted_values = extract_value_and_unit_for_entity(easyocr_result, entity_name)
    if extracted_values:
        return extracted_values[0]

# Example: Run the OCR pipeline in parallel for a batch of images
def process_images_in_parallel(images, entity_name):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda img: ocr_pipeline(img, entity_name), images))
    return results


/nlsasfs/home/aipsc/myksingh/myvenv/include/.venv/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(cop

In [9]:
from concurrent.futures import ThreadPoolExecutor
import threading

# Create a lock for thread-safe access to results
results_lock = threading.Lock()
results = []

def process_image(image, entity_name):
    """Process an individual image and append the result to the shared results list."""
    result = ocr_pipeline(image, entity_name)
    with results_lock:
        results.append(result)

counter = 0
for i in range(len(test_df["entity_name"].values)):
    images = test_generator.__getitem__(counter)

    # Process images in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        # Submit each image-entity pair for parallel processing
        futures = [
            executor.submit(process_image, images[j], test_df["entity_name"].values[j + 16 * counter])
            for j in range(len(images))
        ]
    
    # Wait for all tasks in this batch to complete
    for future in futures:
        future.result()  # This ensures we catch any exceptions that might occur
    
    print(f"Batch {counter} processed")
    counter += 1


Batch 0 processed
Batch 1 processed


/nlsasfs/home/aipsc/myksingh/myvenv/include/.venv/lib/python3.10/site-packages/torch/nn/modules/rnn.py:917: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Batch 2 processed
Batch 3 processed
Batch 4 processed
Batch 5 processed
Batch 6 processed
Batch 7 processed
Batch 8 processed
Batch 9 processed
Batch 10 processed
Batch 11 processed
Batch 12 processed
Batch 13 processed
Batch 14 processed
Batch 15 processed
Batch 16 processed
Batch 17 processed
Batch 18 processed
Batch 19 processed
Batch 20 processed
Batch 21 processed
Batch 22 processed
Batch 23 processed
Batch 24 processed
Batch 25 processed
Batch 26 processed
Batch 27 processed
Batch 28 processed
Batch 29 processed
Batch 30 processed
Batch 31 processed
Batch 32 processed
Batch 33 processed
Batch 34 processed
Batch 35 processed
Batch 36 processed
Batch 37 processed
Batch 38 processed
Batch 39 processed
Batch 40 processed
Batch 41 processed
Batch 42 processed
Batch 43 processed
Batch 44 processed
Batch 45 processed
Batch 46 processed
Batch 47 processed
Batch 48 processed
Batch 49 processed
Batch 50 processed
Batch 51 processed
Batch 52 processed
Batch 53 processed
Batch 54 processed
B